In [1]:
## standart libraries
import os,sys
import numpy as np

# xarray
import xarray as xr

# plota
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap

import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import matplotlib.style as style
style.use('tableau-colorblind10')

In [2]:
from dask.distributed import Client
c = Client()
# c = Client(n_workers=6, threads_per_worker=1, memory_limit="8 GiB",local_directory="/gpfsscratch/rech/cli/uor98hu/dask_tmp/")
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 40.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41385,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 40.00 GiB
Comm: tcp://127.0.0.1:40545,Total threads: 4
Dashboard: http://127.0.0.1:40831/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:33429,


In [3]:
maskBG = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc', chunks = {"x":200,"y":320}).BGRmask.squeeze()

# ------------------------
# loading data to prepare for the volume weighted mean
# ------------------------
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr=xr.open_dataset(diri+'mesh_hgr.nc', chunks = {"x":200,"y":320}).squeeze()
e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

bgr_e1t = e1t.where(maskBG,drop=True)
bgr_e2t = e2t.where(maskBG,drop=True)

prefix = "eORCA025.L75-IMHOTEP"
dirww = "/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/final_after_review/"

chunk_size = {"x":200,"y":320, 'deptht':5}
nbsec = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/nbsec_1m.nc',chunks={'time_counter':1}).nbsec

for nexpREF in ["AI", "S"] : #, "GAI"]:

    dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/"

    diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."+nexpREF+"-S/"
    
    # ------------------------
    # read e3t and compute the volume weight
    # ------------------------
    e3t = xr.open_mfdataset(diridatref+"1m/????/*1m_gridT.nc",decode_coords=False,chunks=chunk_size\
                                ,parallel=True).e3t

    bgr_e3t = e3t.chunk({'time_counter':e3t.shape[0]}).where(maskBG,drop=True)

    # gridcell volume
    btbox = bgr_e3t*bgr_e2t*bgr_e1t
    voltotal = btbox.sum(("deptht","y","x"))

    # ------------------------
    # load adv_h data
    # ------------------------
    print("adv_h open")
    adv_h = xr.open_mfdataset(dirsc +"adv_h/"+nexpREF+"_adv_h_1m*.nc",chunks=chunk_size, parallel = True).adv_h.where(maskBG,drop=True)
    print("adv_h compute")
    adv_h_box_mean_comp = ( (adv_h*btbox).sum(("deptht","y","x")) / voltotal ).compute()

    adv_h_box_mean_cumsum = (adv_h_box_mean_comp*nbsec).cumsum().compute()
    print("adv_h save")
    adv_h_box_mean_cumsum.to_dataset(name='adv_h_cumsum').to_netcdf(path = dirww+nexpREF+"_adv_h_cumsum_1980_2018_after_review.nc", mode='w')


adv_h open
adv_h compute
adv_h save
adv_h open
adv_h compute
adv_h save
